#**📝¡SubtitlesForEveryone!** Transcribe y traduce de forma gratuita.

¡Bienvenido al notebook oficial de **SubtitlesForEveryone**!

>Este Notebook, recopila y facilita el uso de herramientas de inteligencia artificial para la transcripcion y traduccion de audio. Si necesitas crear subtitulos de tu pelicula o de un video, solo tienes que seguir las indicaciones que te proporcionaré a continuación, y comprobarás lo fácil que es.

## ¿Cómo empezar? 🚀

1. **Habilita la GPU**: Para obtener una transcripción rápida, asegúrate de habilitar la GPU. Ve a "Entorno de ejecución" > "Cambiar tipo de entorno de ejecución" > y selecciona "T4 GPU" en la opción "Acelerador de hardware".
2. **Sube tu audio o Pega Una URL de Youtube**: Usa la herramienta de abajo para subir tu archivo de audio o usa la herramienta para importar tu video de youtube.
3. **Ejecuta las celdas**: Simplemente ejecuta las celdas en orden, ¡y verás la magia suceder! ✨

**Nota**: Si eres nuevo en Google Colab, cada celda con código se ejecuta haciendo clic en el botón de reproducción (▶️) a la izquierda de la celda, o puedes presionar `Shift + Enter`.


# **1. ¡Instalación de paquetes necesarios! 📦**
Primero, necesitamos descargar todas las librerias necesarias para hacer funcionar este notebook!

In [ ]:
#Instalacion de paquetes livianos pero necesarios
!pip install srtranslator
!pip3 install ip-rotator
!pip install yt-dlp
!pip install srt
!pip install translators

In [ ]:
#Instalacion de paquetes fundamentales
!pip install demucs
!pip install git+https://github.com/m-bain/whisperx

# **Ahora si, Crea tu subtitulo!** 📜

In [ ]:
#@markdown # **1. Sube tu archivo de audio 📻**
#@markdown Haz clic en el botón de abajo para subir tu archivo de audio. Asegúrate de que sea un archivo en formato MP3.
from google.colab import files

uploaded = files.upload()
audio_file = list(uploaded.keys())[0]

# Renombra el archivo de audio
import os
os.rename(audio_file, "audio.mp3")

In [ ]:
import sys
import warnings
import yt_dlp
import subprocess
import torch
import shutil
import numpy as np
from IPython.display import display, Markdown, YouTubeVideo

device = torch.device('cuda:0')
print('Using device:', device, file=sys.stderr)

Type = "Youtube video or playlist"
#@markdown # Descarga el audio de su video de youtube 📀
URL = "" #@param {type:"string"}
#@markdown Tardara unos segundos en cargar, porfavor, sea paciente.

video_path_local_list = []

if Type == "Youtube video or playlist":

  ydl_opts = {
    'format': 'm4a/bestaudio/best',
    'outtmpl': 'audio',  # Rename the extracted MP3 file to "audio.mp3"
    'postprocessors': [{
      'key': 'FFmpegExtractAudio',
      'preferredcodec': 'mp3',
    }]
  }

  with yt_dlp.YoutubeDL(ydl_opts) as ydl:
    error_code = ydl.download([URL])
    list_video_info = [ydl.extract_info(URL, download=False)]

#change sample rate
import librosa
import soundfile as sf

def convertir_frecuencia(archivo_entrada, archivo_salida):
  # Cargar el archivo de audio
  senal, frecuencia_muestreo = librosa.load(archivo_entrada)

  # Convertir a mono
  senal_mono = librosa.to_mono(senal)

  # Nueva frecuencia de muestreo
  nueva_frecuencia = 16000

  # Convertir la señal a la nueva frecuencia
  senal_convertida = librosa.resample(senal, orig_sr=frecuencia_muestreo, target_sr=nueva_frecuencia)

  # Guardar el archivo de audio convertido
  sf.write(archivo_salida, senal_convertida, nueva_frecuencia)

# Ejemplo de uso
archivo_entrada = "audio.mp3"
archivo_salida = "audio_16_000.mp3"

convertir_frecuencia(archivo_entrada, archivo_salida)

In [ ]:
#@markdown # *Extract vocals From Audio (Alpha)*
#@markdown (Opcional, pero <font color=4bb340> **Altamente Recomendable** <font color=d5d5d5> usarlo):
#@markdown - ¡Gran Mejora En Idiomas Orientales!
#@markdown - Evita que WhisperX Confunda Sonidos de Fondo con el Habla.
#@markdown - Mejora la transcripcion con cualquier idioma.
#@markdown - Gracias a esta extraccion, sera muy raro e improbable que tengas errores en tu transcripcion.

#@markdown **Unico problema:** Aun no se puede usar con audios de una hora. Funciona muy bien con audios de 30 minutos.
!python -m demucs.separate /content/audio_16_000.mp3 --mp3 --mp3-bitrate 128 -n hdemucs_mmi --out /content --mp3-preset 7 -d cuda --two-stems=vocals

In [ ]:
#@markdown # **2. Transcripción de voz ✍️**
#@markdown Ahora, vamos a transcribir tu archivo de audio.
!whisperx /content/audio.mp3 --model large-v3 --output_dir . --device cuda --fp16 True --batch_size 16 --compute_type float16 --beam_size 10 --print_progress True --chunk_size 10 --no_align

### Descargar el resultado 📂
Download_file = False # @param{type:"boolean"}
if Download_file:
 from google.colab import files
 files.download ("audio.srt")

In [ ]:
#@markdown # **3. Traduzca Tu Subtitulo! 🌍**
#@markdown Puedes hacer una transcripcion de tu subtitulo y transformarlo en otro idioma. Puedes usar mas de 6 traductores para elegir.

import ip_rotator
proxy = ip_rotator.Proxy(https = True)
import translators
import srt

# Read the original SRT file
with open("audio.srt", "r", encoding="utf-8") as f:
    subtitles = srt.parse(f.read())

# Function to translate a subtitle (consider error handling)
def translate_subtitle(subtitle):
    try:
        translated_content = translators.translate_text(
            query_text=subtitle.content,
            from_language="auto"#@param {type:"string"}
            ,
            to_language="callate"#@param {type:"string"}
            ,
            translator="deepl" #@param ['alibaba', 'apertium', 'argos', 'baidu', 'bing', 'caiyun', 'cloudTranslation', 'elia', 'google', 'hujiang', 'iciba', 'iflytek', 'iflyrec', 'itranslate', 'judic', 'languageWire', 'lingvanex', 'niutrans', 'mglip', 'mirai', 'modernMt', 'myMemory', 'papago', 'qqFanyi', 'qqTranSmart', 'reverso', 'sogou', 'sysTran', 'tilde', 'translateCom', 'translateMe', 'utibet', 'volcEngine', 'yandex', 'yeekit', 'youdao']{allow-input: true}
        )
        print(f" Subtitile Results {subtitle.index}:")
        print(f"  Original: {subtitle.content}")
        print(f"  Traduccion: {translated_content}")
        return srt.Subtitle(subtitle.index, subtitle.start, subtitle.end, translated_content)
    except Exception as e:  # Catch potential translation errors
        print(f"Error traduciendo subtítulo {subtitle.index}: {e}")
        return subtitle  # Return the original subtitle if translation fails

# Translate each subtitle
translated_subtitles = [translate_subtitle(subtitle) for subtitle in subtitles]

# Write the translated subtitles to a new file
srt_content = srt.compose(translated_subtitles)
with open("translated.srt", "w", encoding="utf-8") as f:
    f.write(srt_content)

### Download result 📂
download_file = False # @param{type:"boolean"}
if download_file:
 from google.colab import files
 files.download ("translated.srt")

In [ ]:
#@markdown # <font color=somblue>**¡DeepL Unlimited Subtitles!** <font color=d5d5d5> *Best Translator.*
#@markdown ¡Traduzca tus subtitulos con gran precision usando DeepL De forma ilimitada!
Source_Language = "auto" # @param ['ar', 'auto', 'bg', 'cs', 'da', 'de', 'el', 'en', 'es', 'et', 'fi', 'fr', 'hu', 'id', 'it', 'ja', 'ko', 'lt', 'lv', 'nb', 'nl', 'pl', 'pt', 'ro', 'ru', 'sk', 'sl', 'sv', 'tr', 'uk', 'zh']{allow-input: true}

#@title Idioma de destino
Target_Language = "es" #@param ['ar', 'bg', 'cs', 'da', 'de', 'el', 'en', 'es', 'et', 'fi', 'fr', 'hu', 'id', 'it', 'ja', 'ko', 'lt', 'lv', 'nb', 'nl', 'pl', 'pt', 'ro', 'ru', 'sk', 'sl', 'sv', 'tr', 'uk', 'zh']{allow-input: true}

import os

# SRT File
from srtranslator import SrtFile

from srtranslator.translators.pydeeplx import PyDeepLX
translator = PyDeepLX()
filepath = "/content/audio.srt"

# SRT File
sub = SrtFile(filepath)
# Translate
sub.translate(translator, Source_Language, Target_Language)

# Making the result subtitles prettier
sub.wrap_lines()

sub.save(f"{os.path.splitext(filepath)[0]}_translated.srt")

### Download result 📂
Download_file = True # @param{type:"boolean"}
if Download_file:
 from google.colab import files
 files.download ("audio_translated.srt")
#@markdown **Advertencia:**
#@markdown *Usar este traductor garantizara traducciones muy buenas, pero no es veloz. El tiempo que dure traduciendo, dependera de lo largo que sea tu subtitulo a traducir.*

## **Lenguajes disponibles para DeepL.** 🗨️


```
'ar': Arabic
'auto': Auto
'bg': Bulgaria
'cs': Czech
'da': Danish
'de': German
'el': Greek
'en': English
'es': Spanish
'et': Estonian
'fi': Finnish
'fr': French
'hu': Hungarian
'id': Indonesian
'it': Italian
'ja': Japanese
'ko': Korean
'lt': Lithuanian
'lv': Latvian
'nb': Norwegian Bokmål
'nl': Dutch
'pl': Polish
'pt': Portuguese
'ro': Romanian
'ru': Russian
'sk': Slovak
'sl': Slovenian
'sv': Swedish
'tr': Turkish
'uk': Ukrainian
'zh': Chinese
```



## **Créditos y Agradecimientos Especiales**
Este notebook, combina muchas librerias gratuitas provenientes de Github y PyPI para lograr hacer su cometido. A continuacion, compartire cuales son:

- [**WhisperX** - Inteligencia Artificial Para transcribir](https://github.com/m-bain/whisperX)
- [**Translators** - Librebria con Api para mas de 10 idiomas](https://pypi.org/project/translators/)
- [**Srt** - Necesario para leer Archivos .srt](https://pypi.org/project/srt/)
- [**Demucs** - Para la extracion de voz](https://pypi.org/project/demucs/)
- [**Ctranslate2** - Necesario para WhisperX](https://pypi.org/project/ctranslate2/)
- [**Yt-dlp** - para la descarga de videos](https://pypi.org/project/yt-dlp/)
- [**Ip-Rotator** - Proxy para Apis](https://pypi.org/project/ip-rotator/)
- [**Srtranslator** - Traduccion De Subtitulos Con Deepl](https://pypi.org/project/srtranslator/)

Aclaracion Importante, este notebook tambien utiliza codigo propio para la creacion de subtitulos; Translators **No fue creado para traducir subtitulos, tuve que escribir yo el codigo para que funcionara de esa forma.** Aun asi, ¡Agradezco enormemente el equipo detrás de todos estos proyectos, son importantes para hacer posible este proyecto! Sin nada mas que decir, Disfruta de este proyecto!